## comapre all


## nothing haappen

In [ ]:
import torch
import torch.nn as nn
import timm
import cv2
import torchvision.transforms as transforms
import torch.nn.functional as F
import mediapipe as mp
import time
import pyautogui


# MediaPipe Hands initialization
mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)


# GPU availability check
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Gesture recognition model path and class names
model_path = "./efficientnet1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "efficientnet_b3a"
num_classes = len(classes_names)

# Initialize the gesture recognition model
class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x


# Function to get PyTorch transforms
def get_torch_transforms(img_size=224):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomRotation((-5, 5)),
            transforms.RandomAutocontrast(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'V' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')



# Initialize the gesture recognition model
model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()

# Initialize webcam capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")

while True:
    # Capture a frame from the webcam
    _, frame = cap.read()
    h, w, c = frame.shape
    if not _:
        break

    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks

    if hand_landmarks:
        # Get timestamp before cropping
        start_time = time.time()
        
        for index, handLMs in enumerate(hand_landmarks):
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h

            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y

            x_min = max(0, x_min - 50)
            y_min = max(0, y_min - 50)
            x_max = min(w, x_max + 50)
            y_max = min(h, y_max + 50)

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

           
            
            cropped_frame = frame[y_min:y_max, x_min:x_max]

            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

            input_tensor = transform(cropped_frame).unsqueeze(0)

            with torch.no_grad():
                output = model(input_tensor)
                probabilities = F.softmax(output[0], dim=0)
                predicted_class = torch.argmax(probabilities).item()
                #use getsure control it
                gesture_control(classes_names[predicted_class])
                #end of it
                predict_name = classes_names[predicted_class] + " for Hand " + str(index + 1)


            cv2.putText(frame, predict_name, (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 0 + index * 100, 0), 2)

            # Get timestamp after processing and before showing the frame
            end_time = time.time()
            # Calculate processing time
            processing_time = end_time - start_time

            print("the process time is "+ str(processing_time))

    else:
        cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)

        #"esc" to quit
    if cv2.waitKey(100) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
eww

the process time is 0.09940934181213379
the process time is 0.07921051979064941
the process time is 0.28438758850097656
the process time is 0.5646357536315918
the process time is 0.2830629348754883
the process time is 0.2856466770172119
the process time is 0.28026366233825684
the process time is 0.2788686752319336
the process time is 0.08571958541870117
the process time is 0.28918027877807617
the process time is 0.2836780548095703
the process time is 0.2809774875640869
the process time is 0.2839949131011963
the process time is 0.28249049186706543
the process time is 0.0841975212097168
the process time is 0.28843164443969727
the process time is 0.08372187614440918
the process time is 0.0887296199798584
the process time is 0.2879514694213867
the process time is 0.2878150939941406
the process time is 0.28119897842407227
the process time is 0.2814309597015381
the process time is 0.2824571132659912
the process time is 0.2883918285369873
the process time is 0.28358936309814453
the process ti

## downsampleing

In [1]:
import torch
import torch.nn as nn
import timm
import cv2
import torchvision.transforms as transforms
import torch.nn.functional as F
import mediapipe as mp
import time
import pyautogui



mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


model_path = "./efficientnet1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "efficientnet_b3a"
num_classes = len(classes_names)


class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x

def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')
    
        #do nothing


def find_bounding_box(landmarks, w, h):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lm in landmarks:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)
    # Add a margin of 50 pixels to the bounding box
    x_min, y_min = max(0, x_min - 50), max(0, y_min - 50)
    x_max, y_max = min(w, x_max + 50), min(h, y_max + 50)
    return x_min, y_min, x_max, y_max

def transform_frame(frame):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(frame).unsqueeze(0)

def classify_gesture(model, input_tensor, classes_names):
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return predicted_class

def display_gesture_prediction(frame, gesture_name, index):
    cv2.putText(frame, f"{gesture_name} for Hand {index + 1}", (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0 + index * 100, 0), 2)


model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")
index = 0

# Downsample factor (skip every N frames)
downsample_factor = 2
downsample_counter = 0
classA="nothing"

while True:

    _, frame = cap.read()
    h, w, c = frame.shape
    if not _:
        break
    # Downsample frames
    if downsample_counter % downsample_factor == 0:
        # Get timestamp before cropping
        start_time = time.time()
        
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            index = 0
            while index < len(hand_landmarks):
                handLMs = hand_landmarks[index]
                x_min, y_min, x_max, y_max = find_bounding_box(handLMs.landmark, w, h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cropped_frame = frame[y_min:y_max, x_min:x_max]
                input_tensor = transform_frame(cropped_frame)
                predicted_class = classify_gesture(model, input_tensor, classes_names)
                gesture_control(classes_names[predicted_class])
                classA= classes_names[predicted_class]
                display_gesture_prediction(frame, classes_names[predicted_class], index)
                index += 1
        else:
            cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        # Get timestamp after processing and before showing the frame
        end_time = time.time()
        # Calculate processing time
        processing_time = end_time - start_time
        print("the process time is "+ str(processing_time))
            
    else:
            gesture_control(classA)

    downsample_counter += 1

cap.release()
cv2.destroyAllWindows()


D:\pytorchMethod\pytorchClassification2\pytorchEnv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


the process time is 0.053270816802978516
the process time is 0.030899763107299805
the process time is 0.030931711196899414
the process time is 0.031241655349731445
the process time is 0.030613183975219727
the process time is 0.031070947647094727
the process time is 0.030059337615966797
the process time is 0.03051018714904785
the process time is 0.03112196922302246
the process time is 0.0307772159576416
the process time is 0.03036665916442871
the process time is 0.03151965141296387
the process time is 0.03053116798400879
the process time is 0.03140568733215332
the process time is 0.03161478042602539
the process time is 0.031122684478759766
the process time is 0.0308685302734375
the process time is 0.03142356872558594
the process time is 0.030759572982788086
the process time is 0.030615806579589844
the process time is 0.03162956237792969
the process time is 0.030982017517089844
the process time is 0.0314030647277832
the process time is 0.02999091148376465
the process time is 0.0307209491

## multithreading

In [ ]:
import cv2
import threading
import queue
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torchvision.transforms as transforms
import mediapipe as mp
import pyautogui
import time

mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_path = "./efficientnet1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "efficientnet_b3a"
num_classes = len(classes_names)


class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x


# Define the gesture recognition function
def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')


# Define the frame processing function
def process_frames():
    while True:
        frame = frame_queue.get()
        h, w, c = frame.shape
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            # Capture timestamp before cropping
            start_time = time.time()
            
            index = 0
            while index < len(hand_landmarks):
                handLMs = hand_landmarks[index]
                x_min, y_min, x_max, y_max = find_bounding_box(handLMs.landmark, w, h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cropped_frame = frame[y_min:y_max, x_min:x_max]
                input_tensor = transform_frame(cropped_frame)
                predicted_class = classify_gesture(model, input_tensor, classes_names)
                gesture_control(classes_names[predicted_class])
                display_gesture_prediction(frame, classes_names[predicted_class], index)
                index += 1



            # Capture timestamp after processing and before showing the frame
            end_time = time.time()
            # Calculate and print processing time
            processing_time = end_time - start_time
            print("the process time is "+ str(processing_time))
        
        else:
            cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break


# Define the helper functions (find_bounding_box, transform_frame, classify_gesture, display_gesture_prediction)


def find_bounding_box(landmarks, w, h):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lm in landmarks:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)
    # Add a margin of 50 pixels to the bounding box
    x_min, y_min = max(0, x_min - 50), max(0, y_min - 50)
    x_max, y_max = min(w, x_max + 50), min(h, y_max + 50)
    return x_min, y_min, x_max, y_max


def transform_frame(frame):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(frame).unsqueeze(0)


def classify_gesture(model, input_tensor, classes_names):
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return predicted_class


def display_gesture_prediction(frame, gesture_name, index):
    cv2.putText(frame, f"{gesture_name} for Hand {index + 1}", (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0 + index * 100, 0), 2)


model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")
index = 0

# Create a queue for passing frames between capture and processing threads
frame_queue = queue.Queue(maxsize=5)

# Create video capture object
cap = cv2.VideoCapture(0)


# Start capture thread
def capture_frames():
    while True:
        _, frame = cap.read()
        if not _:
            break
        frame_queue.put(frame)


# Start processing thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.start()

# Start capture thread
capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

# Wait for threads to finish
capture_thread.join()
processing_thread.join()

# Release video capture object and destroy windows
cap.release()
cv2.destroyAllWindows()


the process time is 0.18364930152893066
the process time is 0.06861138343811035
the process time is 0.2850613594055176
the process time is 0.2715585231781006
the process time is 0.27010512351989746
the process time is 0.558957576751709
the process time is 0.276611328125
the process time is 0.2722659111022949
the process time is 0.27386903762817383
the process time is 0.27530360221862793
the process time is 0.28975367546081543
the process time is 0.27323174476623535
the process time is 0.2790255546569824
the process time is 0.2866668701171875
the process time is 0.2762579917907715
the process time is 0.28015899658203125
the process time is 0.2691612243652344
the process time is 0.2730860710144043
the process time is 0.2801487445831299
the process time is 0.2746121883392334
the process time is 0.2786219120025635
the process time is 0.07120609283447266
the process time is 0.27913546562194824
the process time is 0.27172207832336426
the process time is 0.2702298164367676
the process time is

## resnet50d

In [ ]:
import cv2
import threading
import queue
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torchvision.transforms as transforms
import mediapipe as mp
import pyautogui
import time

mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_path = "./resnet50d_1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "resnet50d"
num_classes = len(classes_names)


class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x


# Define the gesture recognition function
def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')


# Define the frame processing function
def process_frames():
    while True:
        frame = frame_queue.get()
        h, w, c = frame.shape
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            # Capture timestamp before cropping
            start_time = time.time()
            
            index = 0
            while index < len(hand_landmarks):
                handLMs = hand_landmarks[index]
                x_min, y_min, x_max, y_max = find_bounding_box(handLMs.landmark, w, h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cropped_frame = frame[y_min:y_max, x_min:x_max]
                input_tensor = transform_frame(cropped_frame)
                predicted_class = classify_gesture(model, input_tensor, classes_names)
                gesture_control(classes_names[predicted_class])
                display_gesture_prediction(frame, classes_names[predicted_class], index)
                index += 1



            # Capture timestamp after processing and before showing the frame
            end_time = time.time()
            # Calculate and print processing time
            processing_time = end_time - start_time
            print("the process time is "+ str(processing_time))
        
        else:
            cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break


# Define the helper functions (find_bounding_box, transform_frame, classify_gesture, display_gesture_prediction)


def find_bounding_box(landmarks, w, h):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lm in landmarks:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)
    # Add a margin of 50 pixels to the bounding box
    x_min, y_min = max(0, x_min - 50), max(0, y_min - 50)
    x_max, y_max = min(w, x_max + 50), min(h, y_max + 50)
    return x_min, y_min, x_max, y_max


def transform_frame(frame):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(frame).unsqueeze(0)


def classify_gesture(model, input_tensor, classes_names):
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return predicted_class


def display_gesture_prediction(frame, gesture_name, index):
    cv2.putText(frame, f"{gesture_name} for Hand {index + 1}", (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0 + index * 100, 0), 2)


model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")
index = 0

# Create a queue for passing frames between capture and processing threads
frame_queue = queue.Queue(maxsize=5)

# Create video capture object
cap = cv2.VideoCapture(0)


# Start capture thread
def capture_frames():
    while True:
        _, frame = cap.read()
        if not _:
            break
        frame_queue.put(frame)


# Start processing thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.start()

# Start capture thread
capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

# Wait for threads to finish
capture_thread.join()
processing_thread.join()

# Release video capture object and destroy windows
cap.release()
cv2.destroyAllWindows()


the process time is 0.3011937141418457
the process time is 0.08226799964904785
the process time is 0.09833812713623047
the process time is 0.08973240852355957
the process time is 0.2807731628417969
the process time is 0.07016468048095703
the process time is 0.27797722816467285
the process time is 0.2789311408996582
the process time is 0.07518982887268066
the process time is 0.28362298011779785
the process time is 0.2781379222869873
the process time is 0.2785525321960449
the process time is 0.27597570419311523
the process time is 0.2738485336303711
the process time is 0.2911202907562256
the process time is 0.27601099014282227
the process time is 0.27806973457336426
the process time is 0.27904224395751953
the process time is 0.26993227005004883
the process time is 0.27349042892456055
the process time is 0.2761387825012207
the process time is 0.27544569969177246
the process time is 0.27736496925354004
the process time is 0.28522729873657227
the process time is 0.273745059967041
the proces

In [1]:
import torch
import torch.nn as nn
import timm
import cv2
import torchvision.transforms as transforms
import torch.nn.functional as F
import mediapipe as mp
import time
import pyautogui



mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


model_path = "./resnet50d_1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "resnet50d"
num_classes = len(classes_names)


class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x

def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')
    
        #do nothing


def find_bounding_box(landmarks, w, h):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lm in landmarks:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)
    # Add a margin of 50 pixels to the bounding box
    x_min, y_min = max(0, x_min - 50), max(0, y_min - 50)
    x_max, y_max = min(w, x_max + 50), min(h, y_max + 50)
    return x_min, y_min, x_max, y_max

def transform_frame(frame):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(frame).unsqueeze(0)

def classify_gesture(model, input_tensor, classes_names):
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return predicted_class

def display_gesture_prediction(frame, gesture_name, index):
    cv2.putText(frame, f"{gesture_name} for Hand {index + 1}", (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0 + index * 100, 0), 2)


model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")
index = 0

# Downsample factor (skip every N frames)
downsample_factor = 2
downsample_counter = 0
classA="nothing"

while True:

    _, frame = cap.read()
    h, w, c = frame.shape
    if not _:
        break
    # Downsample frames
    if downsample_counter % downsample_factor == 0:
        # Get timestamp before cropping
        start_time = time.time()
        
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            index = 0
            while index < len(hand_landmarks):
                handLMs = hand_landmarks[index]
                x_min, y_min, x_max, y_max = find_bounding_box(handLMs.landmark, w, h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cropped_frame = frame[y_min:y_max, x_min:x_max]
                input_tensor = transform_frame(cropped_frame)
                predicted_class = classify_gesture(model, input_tensor, classes_names)
                gesture_control(classes_names[predicted_class])
                classA= classes_names[predicted_class]
                display_gesture_prediction(frame, classes_names[predicted_class], index)
                index += 1
        else:
            cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        # Get timestamp after processing and before showing the frame
        end_time = time.time()
        # Calculate processing time
        processing_time = end_time - start_time
        print("the process time is "+ str(processing_time))
            
    else:
            gesture_control(classA)

    downsample_counter += 1

cap.release()
cv2.destroyAllWindows()


D:\pytorchMethod\pytorchClassification2\pytorchEnv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


the process time is 0.05387282371520996
the process time is 0.023761272430419922
the process time is 0.03233194351196289
the process time is 0.031242847442626953
the process time is 0.031410932540893555
the process time is 0.031603336334228516
the process time is 0.030266284942626953
the process time is 0.031021595001220703
the process time is 0.030955076217651367
the process time is 0.031164884567260742
the process time is 0.031006336212158203
the process time is 0.03130388259887695
the process time is 0.03177928924560547
the process time is 0.030415058135986328
the process time is 0.03088212013244629
the process time is 0.030794858932495117
the process time is 0.030965805053710938
the process time is 0.03136897087097168
the process time is 0.3585853576660156
the process time is 0.3581109046936035
the process time is 0.6240332126617432
the process time is 0.2956080436706543
the process time is 0.6396834850311279
the process time is 0.10399389266967773
the process time is 0.35625147819

In [2]:
import torch
import torch.nn as nn
import timm
import cv2
import torchvision.transforms as transforms
import torch.nn.functional as F
import mediapipe as mp
import time
import pyautogui


# MediaPipe Hands initialization
mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)


# GPU availability check
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Gesture recognition model path and class names
model_path = "./resnet50d_1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "resnet50d"
num_classes = len(classes_names)

# Initialize the gesture recognition model
class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x


# Function to get PyTorch transforms
def get_torch_transforms(img_size=224):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomRotation((-5, 5)),
            transforms.RandomAutocontrast(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')



# Initialize the gesture recognition model
model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()

# Initialize webcam capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")

while True:
    # Capture a frame from the webcam
    _, frame = cap.read()
    h, w, c = frame.shape
    if not _:
        break

    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks

    if hand_landmarks:
        # Get timestamp before cropping
        start_time = time.time()
        
        for index, handLMs in enumerate(hand_landmarks):
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h

            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y

            x_min = max(0, x_min - 50)
            y_min = max(0, y_min - 50)
            x_max = min(w, x_max + 50)
            y_max = min(h, y_max + 50)

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

           
            
            cropped_frame = frame[y_min:y_max, x_min:x_max]

            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

            input_tensor = transform(cropped_frame).unsqueeze(0)

            with torch.no_grad():
                output = model(input_tensor)
                probabilities = F.softmax(output[0], dim=0)
                predicted_class = torch.argmax(probabilities).item()
                #use getsure control it
                gesture_control(classes_names[predicted_class])
                #end of it
                predict_name = classes_names[predicted_class] + " for Hand " + str(index + 1)


            cv2.putText(frame, predict_name, (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 0 + index * 100, 0), 2)

            # Get timestamp after processing and before showing the frame
            end_time = time.time()
            # Calculate processing time
            processing_time = end_time - start_time

            print("the process time is "+ str(processing_time))

    else:
        cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)

        #"esc" to quit
    if cv2.waitKey(100) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


the process time is 0.28357815742492676
the process time is 0.2803187370300293
the process time is 0.2847874164581299
the process time is 0.5572443008422852
the process time is 0.28423047065734863
the process time is 0.28098392486572266
the process time is 0.5631608963012695
the process time is 0.28581738471984863
the process time is 0.2795882225036621
the process time is 0.2771422863006592
the process time is 0.280531644821167
the process time is 0.5623283386230469
the process time is 0.28491997718811035
the process time is 0.5575873851776123
the process time is 0.2759435176849365
the process time is 0.5524685382843018
the process time is 0.27828431129455566
the process time is 0.28847718238830566
the process time is 0.5675716400146484
the process time is 0.2787160873413086
the process time is 0.5557639598846436
the process time is 0.07411789894104004
the process time is 0.15107011795043945
the process time is 0.08267045021057129
the process time is 0.28304481506347656
the process tim

##  vit_tiny_patch16_224

In [4]:
import torch
import torch.nn as nn
import timm
import cv2
import torchvision.transforms as transforms
import torch.nn.functional as F
import mediapipe as mp
import time
import pyautogui


# MediaPipe Hands initialization
mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)


# GPU availability check
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Gesture recognition model path and class names
model_path = "./vit_tiny_patch16_224_1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "vit_tiny_patch16_224"
num_classes = len(classes_names)

# Initialize the gesture recognition model
class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x


# Function to get PyTorch transforms
def get_torch_transforms(img_size=224):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomRotation((-5, 5)),
            transforms.RandomAutocontrast(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')



# Initialize the gesture recognition model
model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()

# Initialize webcam capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")

while True:
    # Capture a frame from the webcam
    _, frame = cap.read()
    h, w, c = frame.shape
    if not _:
        break

    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks

    if hand_landmarks:
        # Get timestamp before cropping
        start_time = time.time()
        
        for index, handLMs in enumerate(hand_landmarks):
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h

            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y

            x_min = max(0, x_min - 50)
            y_min = max(0, y_min - 50)
            x_max = min(w, x_max + 50)
            y_max = min(h, y_max + 50)

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

           
            
            cropped_frame = frame[y_min:y_max, x_min:x_max]

            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

            input_tensor = transform(cropped_frame).unsqueeze(0)

            with torch.no_grad():
                output = model(input_tensor)
                probabilities = F.softmax(output[0], dim=0)
                predicted_class = torch.argmax(probabilities).item()
                #use getsure control it
                gesture_control(classes_names[predicted_class])
                #end of it
                predict_name = classes_names[predicted_class] + " for Hand " + str(index + 1)


            cv2.putText(frame, predict_name, (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 0 + index * 100, 0), 2)

            # Get timestamp after processing and before showing the frame
            end_time = time.time()
            # Calculate processing time
            processing_time = end_time - start_time

            print("the process time is "+ str(processing_time))

    else:
        cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)

        #"esc" to quit
    if cv2.waitKey(100) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


the process time is 0.031655073165893555
the process time is 0.027419567108154297
the process time is 0.2321782112121582
the process time is 0.03368043899536133
the process time is 0.03470015525817871
the process time is 0.06879162788391113
the process time is 0.031538963317871094
the process time is 0.0357213020324707
the process time is 0.028597116470336914
the process time is 0.0305330753326416
the process time is 0.028260231018066406
the process time is 0.029580354690551758
the process time is 0.26087498664855957
the process time is 0.027692317962646484
the process time is 0.03080272674560547
the process time is 0.026726245880126953
the process time is 0.030825376510620117
the process time is 0.026008129119873047
the process time is 0.2295849323272705
the process time is 0.22879242897033691
the process time is 0.024541616439819336
the process time is 0.031116247177124023
the process time is 0.029590845108032227
the process time is 0.026777267456054688
the process time is 0.02970528

In [5]:
import torch
import torch.nn as nn
import timm
import cv2
import torchvision.transforms as transforms
import torch.nn.functional as F
import mediapipe as mp
import time
import pyautogui



mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


model_path = "./vit_tiny_patch16_224_1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "vit_tiny_patch16_224"
num_classes = len(classes_names)


class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x

def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')
    
        #do nothing


def find_bounding_box(landmarks, w, h):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lm in landmarks:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)
    # Add a margin of 50 pixels to the bounding box
    x_min, y_min = max(0, x_min - 50), max(0, y_min - 50)
    x_max, y_max = min(w, x_max + 50), min(h, y_max + 50)
    return x_min, y_min, x_max, y_max

def transform_frame(frame):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(frame).unsqueeze(0)

def classify_gesture(model, input_tensor, classes_names):
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return predicted_class

def display_gesture_prediction(frame, gesture_name, index):
    cv2.putText(frame, f"{gesture_name} for Hand {index + 1}", (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0 + index * 100, 0), 2)


model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")
index = 0

# Downsample factor (skip every N frames)
downsample_factor = 2
downsample_counter = 0
classA="nothing"

while True:

    _, frame = cap.read()
    h, w, c = frame.shape
    if not _:
        break
    # Downsample frames
    if downsample_counter % downsample_factor == 0:
        # Get timestamp before cropping
        start_time = time.time()
        
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            index = 0
            while index < len(hand_landmarks):
                handLMs = hand_landmarks[index]
                x_min, y_min, x_max, y_max = find_bounding_box(handLMs.landmark, w, h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cropped_frame = frame[y_min:y_max, x_min:x_max]
                input_tensor = transform_frame(cropped_frame)
                predicted_class = classify_gesture(model, input_tensor, classes_names)
                gesture_control(classes_names[predicted_class])
                classA= classes_names[predicted_class]
                display_gesture_prediction(frame, classes_names[predicted_class], index)
                index += 1
        else:
            cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        # Get timestamp after processing and before showing the frame
        end_time = time.time()
        # Calculate processing time
        processing_time = end_time - start_time
        print("the process time is "+ str(processing_time))
            
    else:
            gesture_control(classA)

    downsample_counter += 1

cap.release()
cv2.destroyAllWindows()


the process time is 0.06196236610412598
the process time is 0.030739307403564453
the process time is 0.030701160430908203
the process time is 0.04758858680725098
the process time is 0.03732442855834961
the process time is 0.030889272689819336
the process time is 0.030218839645385742
the process time is 0.031404733657836914
the process time is 0.03101038932800293
the process time is 0.03103923797607422
the process time is 0.030649662017822266
the process time is 0.03205537796020508
the process time is 0.03085923194885254
the process time is 0.03078007698059082
the process time is 0.03075265884399414
the process time is 0.030982017517089844
the process time is 0.03167724609375
the process time is 0.03149819374084473
the process time is 0.030037879943847656
the process time is 0.030988454818725586
the process time is 0.031574249267578125
the process time is 0.03141522407531738
the process time is 0.031339406967163086
the process time is 0.09270715713500977
the process time is 0.1242926120

In [ ]:
import cv2
import threading
import queue
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torchvision.transforms as transforms
import mediapipe as mp
import pyautogui
import time

mphands = mp.solutions.hands
hands = mphands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_path = "./vit_tiny_patch16_224_1.pth"
classes_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
img_size = 224
model_name = "vit_tiny_patch16_224"
num_classes = len(classes_names)


class SELFMODEL(nn.Module):
    def __init__(self, model_name, out_features=num_classes, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if model_name[:3] == "res":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, out_features)
        elif model_name[:3] == "vit":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, out_features)
        else:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, out_features)

    def forward(self, x):
        x = self.model(x)
        return x


# Define the gesture recognition function
def gesture_control(recognized_gestures):
    if 'space' in recognized_gestures:
        pyautogui.keyDown('e')
        pyautogui.keyUp('e')
    elif 'O' in recognized_gestures:
        pyautogui.keyDown('q')
        pyautogui.keyUp('q')
    elif 'N' in recognized_gestures:
        pyautogui.keyDown('w')
        pyautogui.keyUp('w')
    elif 'B' in recognized_gestures:
        pyautogui.keyDown('s')
        pyautogui.keyUp('s')
    elif 'S' in recognized_gestures:
        pyautogui.keyDown('a')
        pyautogui.keyUp('a')
    elif 'F' in recognized_gestures:
        pyautogui.keyDown('d')
        pyautogui.keyUp('d')


# Define the frame processing function
def process_frames():
    while True:
        frame = frame_queue.get()
        h, w, c = frame.shape
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(framergb)
        hand_landmarks = result.multi_hand_landmarks
        if hand_landmarks:
            # Capture timestamp before cropping
            start_time = time.time()
            
            index = 0
            while index < len(hand_landmarks):
                handLMs = hand_landmarks[index]
                x_min, y_min, x_max, y_max = find_bounding_box(handLMs.landmark, w, h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cropped_frame = frame[y_min:y_max, x_min:x_max]
                input_tensor = transform_frame(cropped_frame)
                predicted_class = classify_gesture(model, input_tensor, classes_names)
                gesture_control(classes_names[predicted_class])
                display_gesture_prediction(frame, classes_names[predicted_class], index)
                index += 1



            # Capture timestamp after processing and before showing the frame
            end_time = time.time()
            # Calculate and print processing time
            processing_time = end_time - start_time
            print("the process time is "+ str(processing_time))
        
        else:
            cv2.putText(frame, "nothing", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break


# Define the helper functions (find_bounding_box, transform_frame, classify_gesture, display_gesture_prediction)


def find_bounding_box(landmarks, w, h):
    x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lm in landmarks:
        x, y = int(lm.x * w), int(lm.y * h)
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)
    # Add a margin of 50 pixels to the bounding box
    x_min, y_min = max(0, x_min - 50), max(0, y_min - 50)
    x_max, y_max = min(w, x_max + 50), min(h, y_max + 50)
    return x_min, y_min, x_max, y_max


def transform_frame(frame):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(frame).unsqueeze(0)


def classify_gesture(model, input_tensor, classes_names):
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
    return predicted_class


def display_gesture_prediction(frame, gesture_name, index):
    cv2.putText(frame, f"{gesture_name} for Hand {index + 1}", (10, 30 + 30 * index), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (255, 0 + index * 100, 0), 2)


model = SELFMODEL(model_name=model_name, out_features=num_classes, pretrained=False)
weights = torch.load(model_path, map_location=torch.device('cuda'))
model.load_state_dict(weights)
model.eval()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found or cannot be opened.")
index = 0

# Create a queue for passing frames between capture and processing threads
frame_queue = queue.Queue(maxsize=5)

# Create video capture object
cap = cv2.VideoCapture(0)


# Start capture thread
def capture_frames():
    while True:
        _, frame = cap.read()
        if not _:
            break
        frame_queue.put(frame)


# Start processing thread
processing_thread = threading.Thread(target=process_frames)
processing_thread.start()

# Start capture thread
capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

# Wait for threads to finish
capture_thread.join()
processing_thread.join()

# Release video capture object and destroy windows
cap.release()
cv2.destroyAllWindows()


the process time is 0.2686443328857422
the process time is 0.2940218448638916
the process time is 0.26572227478027344
the process time is 0.27216172218322754
the process time is 0.03658914566040039
the process time is 0.029598474502563477
the process time is 0.039162397384643555
the process time is 0.2352914810180664
the process time is 0.23135161399841309
the process time is 0.24903631210327148
the process time is 0.22946763038635254
the process time is 0.23226690292358398
the process time is 0.2397758960723877
the process time is 0.2298414707183838
the process time is 0.22711873054504395
the process time is 0.22739839553833008
the process time is 0.22806549072265625
the process time is 0.23130011558532715
the process time is 0.22897863388061523
the process time is 0.22956037521362305
the process time is 0.23091888427734375
the process time is 0.2302541732788086
the process time is 0.22874879837036133
the process time is 0.4581434726715088
the process time is 0.45542025566101074
the p